## Time series with Keras

#### Please upvote if you find it useful

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [ ]:
transactions_train = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv",index_col='t_dat',parse_dates=True)

In [ ]:
transactions_train.tail()

In [ ]:
transactions = transactions_train.groupby('t_dat')['price'].sum().reset_index()

In [ ]:
transactions['ds'] = transactions['t_dat']
transactions['y'] = transactions['price']
transactions = transactions[['ds','y']]

In [ ]:
transactions.tail()

In [ ]:
transactions['ds'] = pd.to_datetime(transactions['ds'])

In [ ]:
transactions.set_index('ds', inplace=True)

In [ ]:
results = seasonal_decompose(transactions['y'])

In [ ]:
results.observed.plot(figsize=(12,6))

In [ ]:
results.trend.plot(figsize=(12,6))

In [ ]:
results.seasonal.plot(figsize=(12,6))

In [ ]:
results.resid.plot(figsize=(12,6))

In [ ]:
transactions.info()

In [ ]:
data = transactions[["y"]]
split_time = 600
x_train = data[:split_time]
x_valid = data[split_time:]

In [ ]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(x_train)
scaled_test = scaler.transform(x_valid)


In [ ]:
window_size = 60
n_features = 1
batch_size = 100
generator = TimeseriesGenerator(scaled_train, scaled_train, length=window_size, batch_size=batch_size)

In [ ]:
len(scaled_train)

In [ ]:
X, y= generator[0]

In [ ]:
X.shape

In [ ]:
rm -rf logs

In [ ]:
log_folder = "logs"

In [ ]:
callbacks =  [TensorBoard(log_dir=log_folder),EarlyStopping(patience=10,monitor="mae",mode="min")]

In [ ]:
%load_ext tensorboard

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(window_size, n_features)), 
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])


model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["mse"])

In [ ]:
epochs = 500
history = model.fit(generator,epochs=epochs,callbacks=callbacks)

In [ ]:
eval_batch = scaled_train[-window_size:]

In [ ]:
eval_batch = eval_batch.reshape((1,window_size,n_features))

In [ ]:
eval_batch.shape

In [ ]:
predict_batch = model.predict(eval_batch)

In [ ]:
predict_batch[0]

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-window_size:]
current_batch = first_eval_batch.reshape((1, window_size, n_features))

In [ ]:
current_pred = model.predict(current_batch)[0]

In [ ]:
current_pred

In [ ]:
test_predictions
test_predictions.append(current_pred) 

In [ ]:
test_predictions

In [ ]:
current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
loss_per_epoch = history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-window_size:]
current_batch = first_eval_batch.reshape((1, window_size, n_features))

for i in range(len(x_valid)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
x_valid['Predictions'] = true_predictions

In [ ]:
x_valid.head()

In [ ]:
x_valid.plot(figsize=(10,6))

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid["y"],x_valid["Predictions"]).numpy()

In [ ]:
# %tensorboard --logdir logs

Don't forget to up vote if you find it useful.